In [1]:
%load_ext autoreload
%autoreload 2
import sys
# remove last element from path
import os
from os.path import abspath, join, exists
sys.path.pop()  # preexisting imports path messing up imports
sys.path.append(abspath(join('..')))  # ,'src'
print("\n".join(sys.path))

/home/petrmiculek/Code/kaggle/sentinel_clouds/notebooks
/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/home/petrmiculek/.local/share/virtualenvs/kaggle-DjzpaDb6/lib/python3.10/site-packages
/home/petrmiculek/Code/kaggle/sentinel_clouds


In [9]:
# standard library
from copy import deepcopy
from types import SimpleNamespace
# external
import numpy as np
import torch
from torch.nn import MSELoss, BCELoss, BCEWithLogitsLoss
from torch.nn.functional import sigmoid
from tqdm.notebook import tqdm
from torchinfo import summary
# local
from src.data.dataset import CloudDataset, get_loaders
from src.models.unet import UNet
from src.models.util import evaluate, compute_metrics

In [5]:
path_data = '/mnt/sdb1/code/sentinel2/sample'
dataset_kwargs = {'tile_size': 224, 'crop_pad_mask': 'crop'}
loader_kwargs = {'batch_size': 1, 'num_workers': 0, 'pin_memory': True, 'shuffle': False}
loader = get_loaders(path_data, **dataset_kwargs, **loader_kwargs)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet(in_channels=4).eval().to(device)
# load model weights
path_model = 'model.pt'
model.load_state_dict(torch.load(path_model, map_location=device))
model_summary = summary(model, input_size=(1, 4, 224, 224))
print(model_summary)  # todo shorter summary
criterion = BCEWithLogitsLoss()

Layer (type:depth-idx)                   Output Shape              Param #
UNet                                     [1, 1, 224, 224]          --
├─ModuleList: 1-9                        --                        (recursive)
│    └─DoubleConvBatchNorm: 2-1          [1, 32, 224, 224]         --
│    │    └─Sequential: 3-1              [1, 32, 224, 224]         10,496
├─MaxPool2d: 1-2                         [1, 32, 112, 112]         --
├─ModuleList: 1-9                        --                        (recursive)
│    └─DoubleConvBatchNorm: 2-2          [1, 64, 112, 112]         --
│    │    └─Sequential: 3-2              [1, 64, 112, 112]         55,552
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─ModuleList: 1-9                        --                        (recursive)
│    └─DoubleConvBatchNorm: 2-3          [1, 128, 56, 56]          --
│    │    └─Sequential: 3-3              [1, 128, 56, 56]          221,696
├─MaxPool2d: 1-6                         [1, 

In [10]:
results_val = evaluate(model, loader['test'], criterion, device)
results_val

  0%|          | 0/3 [00:00<?, ?it/s]

{'loss': 0.4350527822971344,
 'labels': array([[[[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          ...,
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]]],
 
 
        [[[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          ...,
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]]],
 
 
        [[[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          ...,
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]]]], dtype=float32),
 'preds': array([[[[0.55247295, 0.5553526 , 0.5614177 , ..., 0.5702827 ,
           0.57017   , 0.5630103 ],
          [0.5583326 , 0.58311385, 0.5

In [12]:
metrics = compute_metrics(results_val['labels'], results_val['preds'])
metrics

{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'F1': 1.0, 'IoU': 1.0}

In [15]:
results_val['labels'].mean(), results_val['preds'].mean()

(1.0, 0.60691595)

In [26]:
model_summary.total_param_bytes, model_summary.total_params
# model_summary.__dict__

(124403204, 31100801)